# Test Description
This script will test the skew between channels and markers of multiple proteus modules.This script checks the channel-to-marker synchronization by changing the SCLK from 1GHz to 9G with fixed jump (can be change) by reading from a Keysight MSO9254A.

NOTE- This script will compare the skew between following-
1. Channel 1 module 1 and Channel 1 Marker 1  module 2
2. Channel 1 module 1 and Channel 1 Marker 2  module 2
3. Channel 3 module 1 and Channel 3 Marker 1  module 2
4. Channel 3 module 1 and Channel 3 Marker 2  module 2

As per datasheet maximum possible skew between channels for multimodule is 300psec. 
Test is getting fail if skew is beyond 300psec.


# Test Equipment
1. Tabor Proteus 9484M
2. Keysight Oscilloscope MSO9254A
3. 4 SMA to BNC Cable
4. SMA to SMP connector



# Hardware Connection

    Proteus Connection  ---> Scope Connection

    Module 1 Channel 1,3 ---> Scope channel 1 ... As per instruction
    Module 2 Marker 1,2 ---> Scope channel 2 ... As per instruction
     

# Test Procedure
1. Connect all the channels of Proteus module with the concerned channel of oscilloscope.
2. Run the script and observe output on scope.



In [1]:
# Import required Python packages.
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt
import sys
import os
srcpath = os.path.realpath('..//..//SourceFiles')
sys.path.append(srcpath)

import pyvisa as visa
from pyvisa.errors import Error
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

# Configurations:
proteus_addr = 'TCPIP::192.90.70.22::5025::SOCKET'  # instruments VISA address of 
#scope_addr =  "USB0::0x2A8D::0x904B::MY55380101::INSTR" # instruments VISA address 
#scope_addr=  "USB0::0x2A8D::0x901C::MY53180102::INSTR"
scope_addr= 'USB0::0x2A8D::0x900E::MY55490134::INSTR'

# connect to proteus via TCP
try:
    inst = TEVisaInst(proteus_addr)
except TEVisaInst.Error as  ex1:
    print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
    sys.exit()

# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr)
        sys.exit()

inst.default_paranoia_level = 2 # Set the default paranoia-level (0, 1 or 2)
inst.send_scpi_cmd('*CLS; *RST') # Clear error-list and reset the instrument
resp = inst.send_scpi_query(":SYST:ERR?")
print(resp)
resp = inst.send_scpi_query('*IDN?') # Get the instrument's *IDN
print('Connected to: ' + resp)
model_name = inst.send_scpi_query('SYST:INF:MODel?') # Get the model:
print('Model: {0} '.format(model_name))

# Get model dependant parameters:
if model_name.startswith('P948'):
    bpp = 2
    dac_mode = 16
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 1
elif model_name.startswith('P908'):
    bpp = 1
    dac_mode = 8
    max_dac = 255
    wpt_type = np.uint8
    offset_factor = 1
else:
    bpp = 2
    dac_mode = 16
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 2




# Infer the natural DAC waveform format
if 'P908' in model_name:
    dac_mode = 8
    MaximumFreqSampleRate=9000
elif 'P94' in model_name:
    dac_mode = 16
else:
    dac_mode = 16
print('DAC waveform format: {} bits-per-point'.format(dac_mode))
max_dac = 2 ** dac_mode - 1
half_dac = max_dac / 2.0
print('Max DAC wave-point level: {}'.format(max_dac))
print('Half DAC wave-point level: {}'.format(half_dac))

# Get number of channels
num_channels = int(inst.send_scpi_query(':INST:CHAN? MAX'))
print('Number of channels: {}'.format(num_channels))

# Get the maximal number of segments
max_seg_number = int(inst.send_scpi_query(':TRACe:SELect:SEGMent? MAX'))
print('Max segment number: {}'.format(max_seg_number))

# Get the available memory in bytes of wavform-data (per DDR):
arbmem_capacity = int(inst.send_scpi_query(':TRACe:FREE?'))
print('Available memory per DDR: {:,} wave-bytes'.format(arbmem_capacity))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Couldn't connect to 'USB0::0x2A8D::0x900E::MY55490134::INSTR', exiting now...
Traceback (most recent call last):
  File "C:\Users\MihirPatel\AppData\Local\Temp\ipykernel_10400\4053309781.py", line 53, in <module>
    scope = resourceManager.open_resource(scope_addr)
  File "C:\Users\MihirPatel\anaconda3\lib\site-packages\pyvisa\highlevel.py", line 3284, in open_resource
    res.open(access_mode, open_timeout)
  File "C:\Users\MihirPatel\anaconda3\lib\site-packages\pyvisa\resources\resource.py", line 282, in open
    self.session, status = self._resource_manager.open_bare_resource(
  File "C:\Users\MihirPatel\anaconda3\lib\site-packages\pyvisa\highlevel.py", line 3209, in open_bare_resource
    return self.visalib.open(self.session, resource_name, access_mode, open_timeout)
  File "C:\Users\MihirPatel\anaconda3\lib\site-packages\pyvisa\ctwrapper\functions.py", line 1851, in open
    ret = library.viOpen(
  File "C:\Users\MihirPatel\anaconda3\lib\site-packages\pyvisa\ctwrapper\highlevel.

TypeError: object of type 'NoneType' has no len()

In [2]:
# Clear and Reset the Modules after selection
inst.send_scpi_cmd(':INST:ACT:SEL 1')
resp = inst.send_scpi_query(":INST:ACT:SEL?")
print("Selected module is" + resp)
inst.send_scpi_cmd('*CLS; *RST')
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)

inst.send_scpi_cmd(':INST:ACT:SEL 2')
resp = inst.send_scpi_query(":INST:ACT:SEL?")
print("Selected module is" + resp)
inst.send_scpi_cmd('*CLS; *RST')
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)

# Clear and Reset the Modules after selection

inst.send_scpi_cmd(':INST:ACT:SEL 1')
resp = inst.send_scpi_query(":INST:ACT:SEL?")
print("Selected module is" + resp)

inst.send_scpi_cmd(':XINS:SYNC:FOLL 0')# 
inst.send_scpi_cmd(':XINS:SYNC:FOLL 1')

inst.send_scpi_cmd(':TRIG:COUPLE ON')
#inst.send_scpi_cmd(':TRIG:CPU:MODE LOCAL')

Selected module is1
Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000002232810,1.238.6 --slot#: 2, slot#: 4, 
Selected module is2
Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000002232810,1.238.6 --slot#: 2, slot#: 4, 
Selected module is1


0

In [3]:
MAX_SKEW = 300e-12
scop_channel1 = 1
scop_channel2 = 4


for ichan in range(1,4,2):
    channb = ichan
    for imark in range(2):
        mark_sel = imark+1

        print('Please connect Channel{} of Module1 with channel1 of scope'.format(channb))
        print('Please connect Channel{0}Marker{1} of Module2 with channel4 of scope and Press Enter'.format(channb,mark_sel))
        
        input()
       

        for freqsamplerate in range(1000,9001,1000):
            inst.send_scpi_cmd(":FREQ:RAST {0}e6".format(freqsamplerate))
            print('\tTest begin for Sampling rate {0}MHz'.format(freqsamplerate))
            
            for instmnt in range(2):
                inst_sel = instmnt+1
                inst.send_scpi_cmd(':INST:ACT:SEL {}'.format(inst_sel))
#                 print('Instrument{} is selected'.format(inst_sel))
                # Build wave-data and markers-data 
                seg_wave_points = 4096
                ncycles =  1
                cyclelen = seg_wave_points/ncycles
                wave = [ None ]
                mark = [ None ]
                # Buffer size for the waveform
                if dac_mode == 16:
                    seg_wave_bytes = seg_wave_points * 2 # each point is 2 bytes in size
                else:
                    seg_wave_bytes = seg_wave_points
                # Buffer size for Marker
                if dac_mode == 16 and freqsamplerate <= 2500:
                    marker_cycle = 2
                    seg_mark_bytes = seg_wave_points // 4
                else:
                    marker_cycle = 8
                    seg_mark_bytes = seg_wave_points // 8
                # Build wave
                x = np.linspace(start=0, stop=seg_wave_points, num=seg_wave_points, endpoint=False)
                yw = np.fmod(x, cyclelen)
                yw = (yw <= cyclelen/2) * max_dac
                yw = np.round(yw)
                yw = np.clip(yw, 0, max_dac)
                yw = yw.astype(wpt_type)
                yw.reshape(-1) # = yw.astype(data_type)
#                 plt.plot(yw)
                # Build marker  
                x = np.linspace(
                    start=0, stop=seg_mark_bytes, num=seg_mark_bytes, endpoint=False)
                y = np.fmod(x, seg_mark_bytes)
                y = (y <= seg_mark_bytes/2) * 0x33
                y = np.round(y)
                y = np.clip(y, 0, 255)
                mark = y
                mark = y.astype(np.uint8).reshape(-1)
                del x, y    
#                 print('Wave and Marker data Created')





                #Download data to Channel
                # Download segments
#                         channb = 1
                segnum = 1
                wav = yw
                mrk = mark
                inst.send_scpi_cmd( ':INST:CHAN {0}'.format(channb))
                inst.send_scpi_cmd(  ':TRAC:DEF {0},{1}'.format(segnum, seg_wave_points))
                inst.send_scpi_cmd(':TRAC:SEL {0}'.format(segnum))
                inst.timeout = 30000
                # Send the binary-data with *OPC? added to the beginning of its prefix.
                #inst.write_binary_values('*OPC?; :TRAC:DATA', wav, datatype=wav_dat_type)
                inst.write_binary_data(':TRAC:DATA', wav)
                # Read the response to the *OPC? query that was added to the prefix of the binary data
                #resp = inst.read()
                # Set normal timeout
                inst.timeout = 10000
                resp = inst.send_scpi_query(':SYST:ERR?')
                resp = resp.rstrip()
                if not resp.startswith('0'):
                    print('ERROR: "{0}" after writing binary values'.format(resp))
                inst.send_scpi_cmd(':INST:CHAN {}'.format(channb))
                inst.send_scpi_cmd('FUNC:MODE ARB')
                # Play the specified segment at the selected channel
                inst.send_scpi_cmd( ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum))
                resp = inst.send_scpi_cmd(':OUTP OFF')
                inst.send_scpi_cmd(':SOUR:VOLT 0.5')
                # Turn on the output of the selected channel:
                resp = inst.send_scpi_cmd(':OUTP ON')
                resp = inst.send_scpi_query(':SYST:ERR?')
                resp = resp.rstrip()
                if not resp.startswith('0'):
                    print('ERROR: "{0}" after writing binary values'.format(resp))
#                 print('Wave data downloaded to channel {0}'.format(channb))




                #Download Marker Data to Markers
                # Increase the timeout before writing binary-data:
                inst.timeout = 30000
                # Send the binary-data with *OPC? added to the beginning of its prefix.
                inst.send_scpi_cmd( ':INST:CHAN {0}'.format(channb))
                inst.send_scpi_cmd( ':TRAC:SEL {0}'.format(segnum))
                inst.write_binary_data(':MARK:DATA',mark)
                inst.send_scpi_cmd(':MARK:SEL {}'.format(mark_sel))
                inst.send_scpi_cmd(':MARK:STAT ON')
                # Read the response to the *OPC? query that was added to the prefix of the binary data
                #resp = inst.read()
                # Set normal timeout
                inst.timeout = 10000
                resp = inst.send_scpi_query(':SYST:ERR?')
                resp = resp.rstrip()
                if not resp.startswith('0'):
                    print('ERROR: "{0}" after writing binary values'.format(resp)) 
#                 print('Marker data downloaded to Marker{}'.format(mark_sel))



                inst.send_scpi_cmd(':INST:ACT:SEL 1')
                resp = inst.send_scpi_query(":INST:ACT:SEL?")
#                 print("Selected module is" + resp)
                inst.send_scpi_cmd(':XINS:SYNC:FOLL 0')
                inst.send_scpi_cmd(':XINS:SYNC:FOLL 1')
                inst.send_scpi_cmd(':TRIG:COUPLE ON')

                
                
            for i in range(4):
                scope.write(':CHANnel{}:DISPlay OFF'.format(i+1))

            scope.write(':CHANnel{}:DISPlay ON'.format(scop_channel1))
            scope.write(':CHANnel{}:DISPlay ON'.format(scop_channel2))
            scope.write(':TIM:SCAL 500E-9')
            scope.write(':TRIGger:EDGE:SOURce CHANnel{}'.format(scop_channel1))
            scope.write(':CHANnel{}:OFFSet 0'.format(scop_channel1))
            time.sleep(1)
            scope.write(':CHANnel{}:OFFSet 0'.format(scop_channel2))
            time.sleep(1)
            scope.write('CHAN{}:SCAL 0.11'.format(scop_channel1))
            time.sleep(1)
            scope.write('CHAN{}:SCAL 0.10'.format(scop_channel2))
            time.sleep(1)
            scope.write(':MEASure:DELTAtime:DEF RISing,1,MIDD,RISing,1,MIDDle')
            time.sleep(2)
            scope.write(':MEASure:DELTAtime CHANnel{},CHANnel{}'.format(scop_channel1,scop_channel2))
            time.sleep(2)
            scope.write(':MEASure:RESults?')
            time.sleep(2)
            resualt = scope.read()
        #     print(resualt)
            mean = float(resualt.split(',')[1])
            tmin = float(resualt.split(',')[2])
            tmax = float(resualt.split(',')[3])
            deviation = float(resualt.split(',')[5])
#             print(abs(mean))


            if (abs(mean)<MAX_SKEW):
                scope.write(':TIM:SCAL 2E-9')
                time.sleep(1)
                scope.write(':MEASure:DELTAtime:DEF RISing,1,MIDD,RISing,1,MIDDle')
                time.sleep(2)
                scope.write(':MEASure:DELTAtime CHANnel{},CHANnel{}'.format(scop_channel1,scop_channel2))
                time.sleep(2)
                scope.write(':MEASure:RESults?')
                resualt = scope.read()
                time.sleep(1)
        #                 print(resualt)
                mean = float(resualt.split(',')[1])
                tmin = float(resualt.split(',')[2])
                tmax = float(resualt.split(',')[3])
                deviation = float(resualt.split(',')[5])
#                 print(abs(mean))

            if (abs(mean)>MAX_SKEW): #200e-12 ):
                print('\t***Test Failed in Sample clock: {}MHz, Mean (Average): {:.2f} ps , Range:[{:.2f}]ps , Jitter: (deviation): {:.2f}ps'.format(freqsamplerate,mean/(10**(-12)),(abs(abs(tmax)-abs(tmin)))/(10**(-12)),deviation/(10**(-12))))
                test_success = False
                #break
            else:
                print('\t***Test Pass in Sample clock: {}MHz, Means(Avrage): {:.2f} ps , Range:[{:.2f}]ps , Jitter: (deviation): {:.2f}ps'.format(freqsamplerate,mean/(10**(-12)),(abs(abs(tmax)-abs(tmin)))/(10**(-12)),deviation/(100)))

print('***All Test Completed for Skew between Channels and Markers for Multimodule')

# close inst connection
inst.close_instrument()
disconnect()

# Close the connection to the instrument
scope.close()
resourceManager.close()
            




Please connect Channel1 of Module1 with channel1 of scope
Please connect Channel1Marker1 of Module2 with channel4 of scope and Press Enter


KeyboardInterrupt: Interrupted by user